In [12]:
from swarmauri.standard.documents.concrete.Document import Document
from swarmauri.standard.vector_stores.concrete.TfidfVectorStore import TfidfVectorStore

In [13]:
# Initialize vector store
vector_store = TfidfVectorStore()

In [14]:
documents = [
    Document(content="He writee Python, a little JavaScript, a little rust, i use dockerfile too"),
    Document(content="He uses sqlite for my database(small projects), json(not really a database) "),
    Document(content="He uses Render, Railway, Northflank and more for deployment"),
    Document(content="He uses docker for containerization"),
    Document(content="His course of study is software engineering"),
    Document(content="He is a Nigerian, currently in Lagos"),
    Document(content="He is from earth, i am not a martian"),
    Document(content="His name is kosisochukwu")
    
]

In [15]:
# Add documents to the vector_store
vector_store.add_documents(documents)

In [16]:
# Verify that they have been added
print(f"{len(vector_store.documents)} documents added to the vector store")

8 documents added to the vector store


In [17]:
# Retrieve and print all documents
all_docs = vector_store.get_all_documents()

print("All documente in te vector store: ")
for doc in all_docs:
    print(doc.content)

All documente in te vector store: 
He writee Python, a little JavaScript, a little rust, i use dockerfile too
He uses sqlite for my database(small projects), json(not really a database) 
He uses Render, Railway, Northflank and more for deployment
He uses docker for containerization
His course of study is software engineering
He is a Nigerian, currently in Lagos
He is from earth, i am not a martian
His name is kosisochukwu


# Configuring the conversation Context

In [18]:
from swarmauri.standard.conversations.concrete.MaxSystemContextConversation import MaxSystemContextConversation
from swarmauri.standard.messages.concrete.SystemMessage import SystemMessage
from swarmauri.standard.messages.concrete.HumanMessage import HumanMessage

In [42]:
# Create a system message
system_context = SystemMessage(content="Your name is Kosisochukwu")

In [43]:
# Initialize the conversation
conversation = MaxSystemContextConversation(system_context=system_context, max_size=4)

In [44]:
# Add a user message
user_message = HumanMessage(content="What is my name?")
conversation.add_message(user_message)

In [45]:
# Print the current conversation context
print("current conversation history")
for message in conversation.history:
    print(message.content)

current conversation history
Your name is Kosisochukwu
What is my name?


## Integrating the LLM

In [46]:
import os
from swarmauri.standard.llms.concrete.GroqModel import GroqModel as LLM
from swarmauri.standard.conversations.concrete.Conversation import Conversation
from dotenv import load_dotenv

# Load envienvironment vaariables
load_dotenv()

True

In [47]:
API_KEY = os.getenv("GROQ_API_KEY")

# check if API key is set 
if not API_KEY:
    print("Please set the API key in the .env g")
else:
    print("API key is set g")

API key is set g


## Function to Get Allowed Models

In [48]:
# Function to gett allowed models filtering out the ones that fail
def get_allowed_models(llm):
    failing_llms=[
        "llama3-70b-8192",
        "llama-3.2-90b-text-preview",
        "mixtral-8x7b-32768",
        "llava-v1.5-7b-4096-preview",
        "llama-guard-3-8b",
    ]
    return [model for model in llm.allowed_models if model not in failing_llms]

In [49]:
# Initialize the GroqModel
llm = LLM(api_key=API_KEY)

In [50]:
# Print model information
print(f"Resource: {llm.resource}")
print(f"Type: {llm.type}")
print(f"Default Name: {llm.name}")

Resource: LLM
Type: GroqModel
Default Name: gemma-7b-it


In [51]:
# Get allowed models
allowed_models = get_allowed_models(llm)
print("Allowed Models: ", allowed_models)

Allowed Models:  ['llama3-8b-8192', 'gemma-7b-it']


In [52]:
# EXAMPLE USAGE WITH NO SYSTEM CONTEXT 

# Set the model name to the first available allowed model
llm.name = allowed_models[0]
print(f"Using model: {llm.name}")

Using model: llama3-8b-8192


In [53]:
# Create a conversation
conversation = Conversation()

In [54]:
# Add a human message
input_data = "Hello"
human_message = HumanMessage(content=input_data)
conversation.add_message(human_message)

In [55]:
# Predict response
llm.predict(conversation=conversation)
prediction =  conversation.get_last().content
print(f"Prediction with no system context for {llm.name}: {prediction}")

Prediction with no system context for llama3-8b-8192: Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?


## Example Usage with a System Context

In [59]:
system_context =  "You are an assistant that proides answers to the user."
conversation = MaxSystemContextConversation(system_context=SystemMessage(content=system_context), max_size=2)

In [60]:
# Create a human message with the content "HI" and add it to the conversation
human_message = HumanMessage(content="Hi")
conversation.add_message(human_message)

In [61]:
# Predict response
llm.predict(conversation=conversation)
prediction = conversation.get_last().content
print(f"Prediction with system context for {llm.name}: {prediction}")

Prediction with system context for llama3-8b-8192: Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?


# BUILDING THE RAG AGENT

In [63]:
from swarmauri.standard.agents.concrete.RagAgent import RagAgent

In [68]:
# Creat a new system context for the rag agent
rag_system_context = "You are an assistant that provides answers to the user. You utilize the details below: "
# Create a new conversation for RAG Agent
rag_conversation = MaxSystemContextConversation(system_context=SystemMessage(content=rag_system_context), max_size=4)

# Initialize the RAG Agent by combining LLM, convrsation, and vector store
rag_agent = RagAgent(
    llm=llm,
    conversation=rag_conversation,
    system_context=rag_system_context,
    vector_store=vector_store,
)

## HANDLING QUERIES WITH THE RAG AGENT

In [69]:
# Test the agent with different queries
queries = [
    "What languages does he write?",
    "What dataabase do they use?",
]

for query in queries:
    response = rag_agent.exec(query)
    print(f"Query: {query}\nRAG AgentResponse: {response}\n")

Query: What languages does he write?
RAG AgentResponse: According to the details, Kosisochukwu writes Python, a little JavaScript, and a little Rust.

Query: What dataabase do they use?
RAG AgentResponse: According to the details, Kosisochukwu uses SQLite for his small projects, and also mentions using JSON, but not really as a database.

